In [3]:
import gym
import math
import time
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
np.random.seed(0)

In [4]:
env = gym.make('MountainCar-v0')

In [5]:
#?env.env

In [6]:
state_space = (10,10)
lower_bounds = [-1.2, -1.5]
upper_bounds = [0.5, 1.5]

def discretizer(pos, velocity):
    est = KBinsDiscretizer(n_bins=state_space, encode='ordinal', strategy='uniform')
    est.fit([lower_bounds, upper_bounds])
    return tuple(map(int,est.transform([[pos, velocity]])[0]))

In [7]:
Q_table = np.zeros(state_space + (env.action_space.n,))
Q_table.shape

(10, 10, 3)

In [8]:
tic = time.time()
Q_table = np.zeros(state_space + (env.action_space.n,))
reward_list = []
total_reward = 0
success = False
for i in range(1, 20000):
    current, done = discretizer(*env.reset()), False
    while not done:
        lr = max(0.01, min(1.0, 1.0 - math.log10((i + 1) / 25)))
        epsilon =  max(0.05, min(1, 1.0 - math.log10((i  + 1) / 25)))
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q_table[current])
        observation, reward, done, info = env.step(action) 
        next_state = discretizer(*observation)
        Q_table[current][action] += lr*(reward + np.max(Q_table[next_state]) - Q_table[current][action])
        current = next_state
        total_reward += reward
#        if i % 100:
#            env.render()
    if i % 100 == 0:
#        print(total_reward/100)
        reward_list.append(total_reward/100)
        if reward_list[-1] >= -110.0:
            toc = time.time()
            success = True
            print(f'Took {toc-tic:.2f} seconds and achieved average reward of {reward_list[-1]} in {(reward_list.index(reward_list[-1]) + 1)*100} episodes')
            env.close()
            break
        total_reward = 0
toc = time.time()
if not success:
    print(f'Took {toc-tic:.2f} seconds and failed')
    print(f'Max average reward achieved: {np.max(reward_list):.2f}')
env.close()
print(Q_table)

Took 1557.76 seconds and failed
Max average reward achieved: -164.75
[[[   0.            0.            0.        ]
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]
  [-269.93338748 -265.24500577 -270.90371561]
  [-262.64262517 -261.15046072 -259.69499683]
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]]

 [[   0.            0.            0.        ]
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]
  [-267.67507978 -266.53499923 -268.02557636]
  [-255.09858408 -255.0708204  -255.09637147]
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]
  [   0.            0.            0.        ]]

 [[   0